In [29]:
import re

In [1]:
!pip install langchain transformers sentence-transformers faiss-cpu PyMuPDF

import fitz  # PyMuPDF
from langchain.docstore.document import Document as LangchainDocument
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 610.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found exist

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = (
    "/content/drive/My Drive/Colab Notebooks/sample_transcript.pdf")

In [4]:
def load_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [5]:
def preprocess_text(text, chunk_size=500):
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    return [LangchainDocument(page_content=chunk) for chunk in chunks]

In [6]:
pdf_text = load_pdf(file_path)
documents = preprocess_text(pdf_text)

In [7]:
documents

[Document(metadata={}, page_content="MachineLearning-Lecture01  \nInstructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is just spend a little time going over the logistics \nof the class, and then we'll start to talk a bit about machine learning.  \nBy way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so \nI personally work in machine learning, and I've worked on it for about 15 years now, and \nI actually think that machine learning is the "),
 Document(metadata={}, page_content="most exciting field of all the computer \nsciences. So I'm actually always excited about teaching this class. Sometimes I actually \nthink that machine learning is not only the most exciting thing in computer science, but \nthe most exciting thing in all of human endeavor, so maybe a little bias there.  \nI also want to introduce the TAs, who are all graduate students doing research in or \nrelated to the machine

In [9]:
pdf_text = load_pdf(file_path)
#documents = preprocess_text(pdf_text)

In [17]:
type(documents)

list

In [22]:
a= re.sub(r"(\w+)-\n(\w+)",r"\1\2", pdf_text) #merge hyphen words

In [ ]:
a

In [26]:
a=re.sub(r"(?<!\n\s)\n(?!\s\n)"," ",a.strip()) #fix new lines in middle of sentences

In [25]:
a

'MachineLearning-Lecture01   Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine  learning class. So what I wanna do today is just spend a little time going over the logistics  of the class, and then we\'ll start to talk a bit about machine learning.   By way of introduction, my name\'s Andrew Ng and I\'ll be instructor for this class. And so  I personally work in machine learning, and I\'ve worked on it for about 15 years now, and  I actually think that machine learning is the most exciting field of all the computer  sciences. So I\'m actually always excited about teaching this class. Sometimes I actually  think that machine learning is not only the most exciting thing in computer science, but  the most exciting thing in all of human endeavor, so maybe a little bias there.   I also want to introduce the TAs, who are all graduate students doing research in or  related to the machine learning and all aspects of machine learning. Paul Baumstarck  works in machine le

In [27]:
a =re.sub(r"\n\s*\n", "\n\n",a) #eliminate multiple lines

In [28]:
a

'MachineLearning-Lecture01   Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine  learning class. So what I wanna do today is just spend a little time going over the logistics  of the class, and then we\'ll start to talk a bit about machine learning.   By way of introduction, my name\'s Andrew Ng and I\'ll be instructor for this class. And so  I personally work in machine learning, and I\'ve worked on it for about 15 years now, and  I actually think that machine learning is the most exciting field of all the computer  sciences. So I\'m actually always excited about teaching this class. Sometimes I actually  think that machine learning is not only the most exciting thing in computer science, but  the most exciting thing in all of human endeavor, so maybe a little bias there.   I also want to introduce the TAs, who are all graduate students doing research in or  related to the machine learning and all aspects of machine learning. Paul Baumstarck  works in machine le

In [30]:
documents = preprocess_text(a)

In [31]:
documents

[Document(metadata={}, page_content="MachineLearning-Lecture01   Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine  learning class. So what I wanna do today is just spend a little time going over the logistics  of the class, and then we'll start to talk a bit about machine learning.   By way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so  I personally work in machine learning, and I've worked on it for about 15 years now, and  I actually think that machine learning is the "),
 Document(metadata={}, page_content="g, and I've worked on it for about 15 years now, and  I actually think that machine learning is the most exciting field of all the computer  sciences. So I'm actually always excited about teaching this class. Sometimes I actually  think that machine learning is not only the most exciting thing in computer science, but  the most exciting thing in all of human endeavor, so maybe a little bias there.   I also want to intr

In [32]:
def preprocess_text_overlap(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return [LangchainDocument(page_content=chunk) for chunk in chunks]

In [33]:
documents = preprocess_text_overlap(a)

In [34]:
len(documents)

151

In [35]:
documents

[Document(metadata={}, page_content="MachineLearning-Lecture01   Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine  learning class. So what I wanna do today is just spend a little time going over the logistics  of the class, and then we'll start to talk a bit about machine learning.   By way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so  I personally work in machine learning, and I've worked on it for about 15 years now, and  I actually think that machine learning is the "),
 Document(metadata={}, page_content="g, and I've worked on it for about 15 years now, and  I actually think that machine learning is the most exciting field of all the computer  sciences. So I'm actually always excited about teaching this class. Sometimes I actually  think that machine learning is not only the most exciting thing in computer science, but  the most exciting thing in all of human endeavor, so maybe a little bias there.   I also want to intr

In [37]:
pdf_text= re.sub(r'\s+', ' ', pdf_text) #multiple spaces

In [38]:
documents = preprocess_text_overlap(pdf_text) #re.sub(r'\s+', ' ', documents)

In [39]:
documents

[Document(metadata={}, page_content="MachineLearning-Lecture01 Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine learning class. So what I wanna do today is just spend a little time going over the logistics of the class, and then we'll start to talk a bit about machine learning. By way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so I personally work in machine learning, and I've worked on it for about 15 years now, and I actually think that machine learning is the most exc"),
 Document(metadata={}, page_content="I've worked on it for about 15 years now, and I actually think that machine learning is the most exciting field of all the computer sciences. So I'm actually always excited about teaching this class. Sometimes I actually think that machine learning is not only the most exciting thing in computer science, but the most exciting thing in all of human endeavor, so maybe a little bias there. I also want to introduce the TAs

In [40]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode([doc.page_content for doc in documents])

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [43]:
embeddings

array([[-0.06366199, -0.03773705,  0.03438343, ...,  0.01462436,
        -0.09145053,  0.00525171],
       [-0.06503486, -0.02758935,  0.04116147, ...,  0.02455043,
        -0.06573199,  0.0444827 ],
       [-0.0374782 , -0.02516202,  0.01503962, ...,  0.06351869,
         0.0324683 , -0.01795302],
       ...,
       [-0.05293226,  0.03901056,  0.0899864 , ...,  0.08066501,
        -0.13496405,  0.07613472],
       [ 0.0138198 ,  0.02470315, -0.00669015, ...,  0.04069093,
        -0.12748629,  0.00499933],
       [-0.00711033, -0.03230452,  0.00465381, ...,  0.00669094,
        -0.0397682 , -0.05913039]], dtype=float32)

In [42]:
embeddings.shape

(149, 384)

In [44]:
def search(query, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [documents[i] for i in indices[0]]

In [45]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [46]:
qa_pipeline

In [47]:
def answer_query(query):
    relevant_docs = search(query)
    context = " ".join([doc.page_content for doc in relevant_docs])
    result = qa_pipeline(question=query, context=context)
    return result['answer']

In [48]:
query = "For how many years Andrew worked in ML"
response = answer_query(query)
response


'15 years'

In [49]:
query = "Who are the TA's for the course"
response = answer_query(query)
response


'Zico Kolter'

In [50]:
def answer_queries(queries):
    responses = []
    for query in queries:
        relevant_docs = search(query)
        context = " ".join([doc.page_content for doc in relevant_docs])
        result = qa_pipeline(question=query, context=context)
        responses.append(result['answer'])
    return responses

In [57]:
queries = ["For how many years Andrew worked in ML", "Who are the TA's for the course", "How many students registered for course?"]
responses = answer_queries(queries)

for i, response in enumerate(responses):
    print(f"Response to query {i+1}: {response}")

Response to query 1: 15 years
Response to query 2: Zico Kolter
Response to query 3: up to three people


In [59]:
queries = ["What is the grading system for this course.", "List all the TA's for the course", "What are the comments about Netflix?"]
responses = answer_queries(queries)

for i, response in enumerate(responses):
    print(f"Response to query {i+1}: {response}")

Response to query 1: MATLAB
Response to query 2: Zico Kolter is the head TA
Response to query 3: these are other examples of learning algorithms


In [60]:
query = "What are the comments about Netflix? Answer only if you are confident about the answer"
response = answer_query(query)
response

'recommend books for you to buy or movies for you to rent or whatever'